In [346]:
!pip install category_encoders
!pip install feature-engine
!pip install catboost

In [347]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravmalik26/food-delivery-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/gauravmalik26/food-delivery-dataset/versions/1


In [348]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer,MissingIndicator, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV,RepeatedStratifiedKFold,cross_val_score,cross_val_predict
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer,make_column_selector,ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from sklearn.impute import  KNNImputer
from sklearn.svm import SVC
from lightgbm import LGBMRegressor, LGBMClassifier
import xgboost as xgb

import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
# from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import holidays
import gc
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from geopy.distance import geodesic

import gc
import joblib


from category_encoders import OrdinalEncoder


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer,LabelEncoder
from feature_engine.encoding import OrdinalEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.encoding import RareLabelEncoder
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from feature_engine.selection import (RecursiveFeatureElimination, RecursiveFeatureAddition,
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,DropCorrelatedFeatures
)
from feature_engine.creation import MathFeatures,RelativeFeatures

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import MeanEncoder
from sklearn import metrics
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.cluster import KMeans
from category_encoders import MEstimateEncoder,OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import KBinsDiscretizer


from sklearn.ensemble import StackingRegressor

In [349]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor ,AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, ElasticNet, LassoLars, PassiveAggressiveRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, metrics, Pool, cv
pd.set_option('display.max_columns', None)

## Data loading

In [350]:
train = pd.read_csv(f"{path}/train.csv")

1. **ID**: Mã định danh duy nhất cho từng đơn hàng.
2. **Delivery_person_ID**: Mã định danh duy nhất của người giao hàng thực hiện đơn hàng.
3. **Delivery_person_Age**: Tuổi của người giao hàng (dạng chuỗi, cần chuyển thành kiểu số nếu muốn phân tích).
4. **Delivery_person_Ratings**: Xếp hạng (rating) của người giao hàng, có thể là trung bình từ các đánh giá của khách hàng (dạng chuỗi, cần chuyển thành kiểu số nếu phân tích).
5. **Restaurant_latitude**: Vĩ độ của nhà hàng nơi đơn hàng được đặt.
6. **Restaurant_longitude**: Kinh độ của nhà hàng nơi đơn hàng được đặt.
7. **Delivery_location_latitude**: Vĩ độ của địa điểm giao hàng.
8. **Delivery_location_longitude**: Kinh độ của địa điểm giao hàng.
9. **Order_Date**: Ngày đặt hàng (dạng chuỗi, có thể chuyển sang định dạng ngày tháng để phân tích thời gian).
10. **Time_Orderd**: Thời gian đơn hàng được đặt (dạng chuỗi, có thể chuyển sang thời gian để phân tích).
11. **Time_Order_picked**: Thời gian người giao hàng nhận hàng từ nhà hàng.
12. **Weatherconditions**: Mô tả điều kiện thời tiết khi đơn hàng được giao (ví dụ: nắng, mưa, sương mù).
13. **Road_traffic_density**: Mô tả mức độ giao thông trên đường khi đơn hàng được giao (ví dụ: thấp, trung bình, cao, kẹt xe).
14. **Vehicle_condition**: Tình trạng phương tiện của người giao hàng, được đánh giá theo thang điểm (kiểu số nguyên).
15. **Type_of_order**: Loại đơn hàng được đặt (ví dụ: đồ ăn, đồ uống, đồ ăn nhanh).
16. **Type_of_vehicle**: Loại phương tiện được sử dụng để giao hàng (ví dụ: xe máy, ô tô).
17. **multiple_deliveries**: Số lượng đơn hàng mà người giao hàng thực hiện cùng một lúc (dạng chuỗi, có thể chuyển sang kiểu số).
18. **Festival**: Có lễ hội diễn ra vào ngày giao hàng hay không (các giá trị: Yes hoặc No).
19. **City**: Thành phố nơi đơn hàng được giao (ví dụ: đô thị, bán đô thị, vùng nông thôn).
20. **Time_taken(min)**: Thời gian giao hàng tính bằng phút (dạng chuỗi, có thể chuyển thành kiểu số để phân tích).

## DATA CLEANING

In [351]:
train=train.replace('NaN', float(np.nan), regex=True)

In [352]:
train['Weatherconditions']=train['Weatherconditions'].str.split(" ", expand=True)[1]
train['Time_taken(min)']=train['Time_taken(min)'].str.split(" ", expand=True)[1]

In [353]:
num_cols = ['Delivery_person_Age',
            'Delivery_person_Ratings',
            'Restaurant_latitude',
            'Restaurant_longitude',
            'Delivery_location_latitude',
            'Delivery_location_longitude',
            'Vehicle_condition',
            'multiple_deliveries',
            'Time_taken(min)']
for col in num_cols:
    train[col]=train[col].astype('float64')
train['Order_Date']=pd.to_datetime(train['Order_Date'],format="%d-%m-%Y")

In [354]:
train['Time_Orderd']=pd.to_timedelta(train['Time_Orderd'])
train['Time_Order_picked']=pd.to_timedelta(train['Time_Order_picked'])

train['Time_Order_picked_formatted']=np.where(train['Time_Order_picked'] < train['Time_Orderd'], train['Order_Date'] + pd.DateOffset(1)+train['Time_Order_picked'], train['Order_Date']+train['Time_Order_picked'])
train['Time_Ordered_formatted'] = train['Order_Date']+ train['Time_Orderd']
train['order_prepare_time_diff_mins']=((train['Time_Order_picked_formatted']- train['Time_Ordered_formatted']).dt.total_seconds())/60

In [355]:
cols=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']
for col in cols:
    train[col]= abs(train[col])

In [356]:
train['distance_diff_KM']=np.zeros(len(train))
restaurant_cordinates_train=train[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
delivery_location_cordinates_train=train[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

for i in range(len(train)):
    train['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_train[i],delivery_location_cordinates_train[i])

In [357]:
y=train["Time_taken(min)"]
train = train.drop(columns=["Time_taken(min)",'ID'],axis=1)

In [358]:
data = pd.concat([train]).reset_index(drop=True)

In [359]:
na_cols=[]
for col in data.columns:
    if data[col].isna().sum()>0:
        na_cols.append(col)

data["n_missing"]= data[na_cols].isna().sum(axis=1)
data_missing_tag_df = data[na_cols].isna().astype(np.int8)
data_missing_tag_df.columns = [f"{c}_missing" for c in data_missing_tag_df.columns]

data=pd.concat([data, data_missing_tag_df], axis=1)

In [360]:
data["day"] = data.Order_Date.dt.day
data["week"] = data.Order_Date.dt.isocalendar().week
data["month"] = data.Order_Date.dt.month
data["quarter"] = data.Order_Date.dt.quarter
data["year"] = data.Order_Date.dt.year
data["hour"] = data.Time_Orderd.dt.components['hours']
data["dayofyear"] = data.Order_Date.dt.dayofyear
data['day_of_week'] = data.Order_Date.dt.day_of_week.astype(int)
data["is_month_start"] = data.Order_Date.dt.is_month_start.astype(int)
data["is_month_end"] = data.Order_Date.dt.is_month_end.astype(int)
data["is_quarter_start"] = data.Order_Date.dt.is_quarter_start.astype(int)
data["is_quarter_end"] = data.Order_Date.dt.is_quarter_end.astype(int)
data["is_year_start"] = data.Order_Date.dt.is_year_start.astype(int)
data["is_year_end"] = data.Order_Date.dt.is_year_end.astype(int)
data["is_leap_year"] = data.Order_Date.dt.is_leap_year.astype(int)
data["days_in_month"] = data.Order_Date.dt.days_in_month
data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]),1,0)

In [361]:
data['distance_diff_KM']=data['distance_diff_KM'].astype("str").str.extract('(\d+)')
data['distance_diff_KM']=data['distance_diff_KM'].astype("int64")

In [362]:
data=data.drop(columns=['Order_Date','Time_Orderd','Time_Order_picked'],axis=1)
data=data.drop(columns=['Time_Order_picked_formatted','Time_Ordered_formatted'])

In [363]:
data['week']=data['week'].astype("int64")

In [364]:
for col in data.columns:
    if data[col].dtype == 'uint8':
        data[col]=data[col].astype("int64")

In [365]:
data['city_code']=data['Delivery_person_ID'].str.split("RES", expand=True)[0]

In [366]:
data=data.drop(columns=["Delivery_person_ID" ],  axis=1)

In [367]:
pipe = Pipeline([
   ('constant', DropConstantFeatures(tol=0.998,missing_values='ignore')),
   ('duplicated', DropDuplicateFeatures(missing_values='ignore')),
])
data=pipe.fit_transform(data)

In [368]:
num_ft = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
mf=MathFeatures(['Delivery_person_Age','Delivery_person_Ratings'],missing_values='ignore',func= ["std",'mean'])
data=mf.fit_transform(data)

feat1=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("mean")
data['feat2']=data['Delivery_person_Ratings'] - feat1

feat3=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("std")
data['feat4']=data['Delivery_person_Ratings'] - feat3

data['feat9']=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("mean")
#data['feat10']=data['Delivery_person_Age'] - data['feat9']

feat11=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("std")
data['feat12']=data['Delivery_person_Age'] - feat11

data['feat13']=data.groupby(["Weatherconditions",'Road_traffic_density','City','city_code'])["distance_diff_KM"].transform("std")
data['feat14']=data['Delivery_person_Age'] - data['feat13']

In [369]:
xgb_model=xgb.XGBRegressor(n_estimators=25,max_depth=7)
lgb = LGBMRegressor(num_leaves=85,max_depth=8,learning_rate=0.1)
cat= CatBoostRegressor(n_estimators=700,max_depth=8,learning_rate=0.05)

In [370]:
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# sfs = SFS(LGBMRegressor(num_leaves=85,max_depth=8,learning_rate=0.1),
#            k_features=20,
#            forward=True,
#            floating=False,
#            verbose=2,
#            scoring='r2',
#            cv=2)

# sfs = sfs.fit(train ,y)
# selected_feat_= list(sfs.k_feature_names_)

# selected_feat_

In [371]:
selected_feat_ =[
  'Delivery_person_Ratings',
 'Weatherconditions',
 'Road_traffic_density',
 'Vehicle_condition',
 'multiple_deliveries',
 'Festival',
 'City',
 'distance_diff_KM',
 'Delivery_person_Age_missing',
 'Delivery_person_Ratings_missing',
 'Time_Orderd_missing',
 'Weatherconditions_missing',
 'Road_traffic_density_missing',
 'multiple_deliveries_missing',
 'Festival_missing',
 'City_missing',
 'days_in_month',
 'is_weekend',
 'feat9',
 'feat12']

In [372]:
imp_feat=selected_feat_[:12]

In [373]:
data = data[imp_feat]

In [374]:
encoder = MEstimateEncoder(m=1)
train= data[:len(train)]
train =  encoder.fit_transform(train,y)
joblib.dump(encoder, 'mestimate_encoder.pkl')

['mestimate_encoder.pkl']

In [375]:
X = train  # Các đặc trưng
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [376]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36474 entries, 29044 to 15795
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Delivery_person_Ratings          34954 non-null  float64
 1   Weatherconditions                36474 non-null  float64
 2   Road_traffic_density             36474 non-null  float64
 3   Vehicle_condition                36474 non-null  float64
 4   multiple_deliveries              35673 non-null  float64
 5   Festival                         36474 non-null  float64
 6   City                             36474 non-null  float64
 7   distance_diff_KM                 36474 non-null  int64  
 8   Delivery_person_Age_missing      36474 non-null  int8   
 9   Delivery_person_Ratings_missing  36474 non-null  int8   
 10  Time_Orderd_missing              36474 non-null  int8   
 11  Weatherconditions_missing        36474 non-null  int8   
dtypes: float64(7), int6

In [377]:
xgb_model.fit(X_train, y_train) # XGBoost
lgb.fit(X_train, y_train) # LightGBM
cat.fit(X_train, y_train) # CatBoost

estimators = [
    ('xgb_model', xgb_model),
    ('lgb', lgb),
    ('cat', cat)
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=xgb_model
)
reg.fit(X_train, y_train)


joblib.dump(reg, 'regression_model.pkl')    # Linear Regression
joblib.dump(lgb, 'lightgbm_model.pkl')      # LightGBM
joblib.dump(cat, 'catboost_model.pkl')      # CatBoost
joblib.dump(xgb_model, 'xgboost_model.pkl') # XGBoost

Streaming output truncated to the last 5000 lines.
204:	learn: 4.7272306	total: 1.56s	remaining: 3.76s
205:	learn: 4.7270798	total: 1.56s	remaining: 3.75s
206:	learn: 4.7246710	total: 1.57s	remaining: 3.74s
207:	learn: 4.7242582	total: 1.58s	remaining: 3.73s
208:	learn: 4.7239008	total: 1.59s	remaining: 3.73s
209:	learn: 4.7234042	total: 1.59s	remaining: 3.72s
210:	learn: 4.7223895	total: 1.6s	remaining: 3.71s
211:	learn: 4.7217182	total: 1.61s	remaining: 3.7s
212:	learn: 4.7213273	total: 1.61s	remaining: 3.69s
213:	learn: 4.7206970	total: 1.62s	remaining: 3.69s
214:	learn: 4.7199616	total: 1.63s	remaining: 3.68s
215:	learn: 4.7196038	total: 1.64s	remaining: 3.67s
216:	learn: 4.7190255	total: 1.65s	remaining: 3.66s
217:	learn: 4.7188274	total: 1.65s	remaining: 3.65s
218:	learn: 4.7181659	total: 1.66s	remaining: 3.65s
219:	learn: 4.7170032	total: 1.67s	remaining: 3.64s
220:	learn: 4.7168472	total: 1.68s	remaining: 3.63s
221:	learn: 4.7165468	total: 1.7s	remaining: 3.65s
222:	learn: 4.71

['xgboost_model.pkl']

In [378]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Đánh giá trên tập train
reg_pred_train = reg.predict(train[imp_feat])
lgb_pred_train = lgb.predict(train[imp_feat])
cat_pred_train = cat.predict(train[imp_feat])
xgb_pred_train = xgb_model.predict(train[imp_feat])

# Tính trung bình ensemble trên tập train
ensemble_train = reg_pred_train * 0.2 + lgb_pred_train * 0.45 + cat_pred_train * 0.35

# Đánh giá trên tập validation
reg_pred_val = reg.predict(X_val[imp_feat])
lgb_pred_val = lgb.predict(X_val[imp_feat])
cat_pred_val = cat.predict(X_val[imp_feat])
xgb_pred_val = xgb_model.predict(X_val[imp_feat])

# Tính trung bình ensemble trên tập val
ensemble_val = reg_pred_val * 0.2 + lgb_pred_val * 0.45 + cat_pred_val * 0.35


# Đánh giá từng mô hình trên tập train
print("Train Performance:")
print("Linear Regression - MAE:", mean_absolute_error(y, reg_pred_train))
print("Linear Regression - RMSE:", np.sqrt(mean_squared_error(y, reg_pred_train)))
print("Linear Regression - R2 Score:", r2_score(y, reg_pred_train))
print()

print("LightGBM - MAE:", mean_absolute_error(y, lgb_pred_train))
print("LightGBM - RMSE:", np.sqrt(mean_squared_error(y, lgb_pred_train)))
print("LightGBM - R2 Score:", r2_score(y, lgb_pred_train))
print()

print("CatBoost - MAE:", mean_absolute_error(y, cat_pred_train))
print("CatBoost - RMSE:", np.sqrt(mean_squared_error(y, cat_pred_train)))
print("CatBoost - R2 Score:", r2_score(y, cat_pred_train))
print()

print("XGBoost - MAE:", mean_absolute_error(y, xgb_pred_train))
print("XGBoost - RMSE:", np.sqrt(mean_squared_error(y, xgb_pred_train)))
print("XGBoost - R2 Score:", r2_score(y, xgb_pred_train))
print()

print("Ensemble - MAE:", mean_absolute_error(y, ensemble_train))
print("Ensemble - RMSE:", np.sqrt(mean_squared_error(y, ensemble_train)))
print("Ensemble - R2 Score:", r2_score(y, ensemble_train))

# Đánh giá từng mô hình trên tập validation
print("\nValidation Performance:")
print("Linear Regression - MAE:", mean_absolute_error(y_val, reg_pred_val))
print("Linear Regression - RMSE:", np.sqrt(mean_squared_error(y_val, reg_pred_val)))
print("Linear Regression - R2 Score:", r2_score(y_val, reg_pred_val))
print()

print("LightGBM - MAE:", mean_absolute_error(y_val, lgb_pred_val))
print("LightGBM - RMSE:", np.sqrt(mean_squared_error(y_val, lgb_pred_val)))
print("LightGBM - R2 Score:", r2_score(y_val, lgb_pred_val))
print()

print("CatBoost - MAE:", mean_absolute_error(y_val, cat_pred_val))
print("CatBoost - RMSE:", np.sqrt(mean_squared_error(y_val, cat_pred_val)))
print("CatBoost - R2 Score:", r2_score(y_val, cat_pred_val))
print()

print("XGBoost - MAE:", mean_absolute_error(y_val, xgb_pred_val))
print("XGBoost - RMSE:", np.sqrt(mean_squared_error(y_val, xgb_pred_val)))
print("XGBoost - R2 Score:", r2_score(y_val, xgb_pred_val))
print()

print("Ensemble - MAE:", mean_absolute_error(y_val, ensemble_val))
print("Ensemble - RMSE:", np.sqrt(mean_squared_error(y_val, ensemble_val)))
print("Ensemble - R2 Score:", r2_score(y_val, ensemble_val))
print()

Train Performance:
Linear Regression - MAE: 3.676959811003072
Linear Regression - RMSE: 4.698621799848409
Linear Regression - R2 Score: 0.7492779965272384

LightGBM - MAE: 3.659907566901845
LightGBM - RMSE: 4.663979146289895
LightGBM - R2 Score: 0.752961483470888

CatBoost - MAE: 3.602727330702782
CatBoost - RMSE: 4.599679552154414
CatBoost - R2 Score: 0.7597260854516046

XGBoost - MAE: 3.6671448183962267
XGBoost - RMSE: 4.680662664053261
XGBoost - R2 Score: 0.7511909597158901

Ensemble - MAE: 3.631933441527492
Ensemble - RMSE: 4.632868431019013
Ensemble - R2 Score: 0.7562461945681225

Validation Performance:
Linear Regression - MAE: 3.799281496269058
Linear Regression - RMSE: 4.8261862003212554
Linear Regression - R2 Score: 0.7343457090994405

LightGBM - MAE: 3.7830340097891866
LightGBM - RMSE: 4.793876246045712
LightGBM - R2 Score: 0.7378907636519959

CatBoost - MAE: 3.7952850858230165
CatBoost - RMSE: 4.810152981026975
CatBoost - R2 Score: 0.7361078535182541

XGBoost - MAE: 3.789318

In [ ]:
# test['Weatherconditions']=test['Weatherconditions'].str.split(" ", expand=True)[1]

In [ ]:
# num_cols_test = ['Delivery_person_Age',
#             'Delivery_person_Ratings',
#             'Restaurant_latitude',
#             'Restaurant_longitude',
#             'Delivery_location_latitude',
#             'Delivery_location_longitude',
#             'Vehicle_condition',
#             'multiple_deliveries',]
# for col in num_cols_test:
#     test[col]=test[col].astype('float64')
# test['Order_Date']=pd.to_datetime(test['Order_Date'],format="%d-%m-%Y")

In [ ]:
# test['Time_Orderd']=pd.to_timedelta(test['Time_Orderd'])
# test['Time_Order_picked']=pd.to_timedelta(test['Time_Order_picked'])

# test['Time_Order_picked_formatted']=np.where(test['Time_Order_picked'] < test['Time_Orderd'], test['Order_Date'] + pd.DateOffset(1)+test['Time_Order_picked'], test['Order_Date']+test['Time_Order_picked'])
# test['Time_Ordered_formatted'] = test['Order_Date']+ test['Time_Orderd']
# test['order_prepare_time_diff_mins']=((test['Time_Order_picked_formatted']- test['Time_Ordered_formatted']).dt.total_seconds())/60

In [ ]:
# cols=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']
# for col in cols:
#     test[col]= abs(test[col])

In [ ]:
# test['distance_diff_KM']=np.zeros(len(test))
# restaurant_cordinates_test=test[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
# delivery_location_cordinates_test=test[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

# for i in range(len(test)):
#     test['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_test[i],delivery_location_cordinates_test[i])

In [ ]:
# test["day"] = test.Order_Date.dt.day
# test["week"] = test.Order_Date.dt.isocalendar().week
# test["month"] = test.Order_Date.dt.month
# test["quarter"] = test.Order_Date.dt.quarter
# test["year"] = test.Order_Date.dt.year
# test["hour"] = test.Time_Orderd.dt.components['hours']
# test["dayofyear"] = test.Order_Date.dt.dayofyear
# test['day_of_week'] = test.Order_Date.dt.day_of_week.astype(int)
# test["is_month_start"] = test.Order_Date.dt.is_month_start.astype(int)
# test["is_month_end"] = test.Order_Date.dt.is_month_end.astype(int)
# test["is_quarter_start"] = test.Order_Date.dt.is_quarter_start.astype(int)
# test["is_quarter_end"] = test.Order_Date.dt.is_quarter_end.astype(int)
# test["is_year_start"] = test.Order_Date.dt.is_year_start.astype(int)
# test["is_year_end"] = test.Order_Date.dt.is_year_end.astype(int)
# test["is_leap_year"] = test.Order_Date.dt.is_leap_year.astype(int)
# test["days_in_month"] = test.Order_Date.dt.days_in_month
# test['is_weekend'] = np.where(test['day_of_week'].isin([5,6]),1,0)

In [ ]:
# test = test.drop(columns=["ID"],axis=1)

In [ ]:
# test['distance_diff_KM']=test['distance_diff_KM'].astype("str").str.extract('(\d+)')
# test['distance_diff_KM']=test['distance_diff_KM'].astype("int64")

In [ ]:
# test=test.drop(columns=['Order_Date','Time_Orderd','Time_Order_picked'],axis=1)
# test=test.drop(columns=['Time_Order_picked_formatted','Time_Ordered_formatted'])

In [ ]:
# test['week']=test['week'].astype("int64")

In [ ]:
# for col in test.columns:
#     if test[col].dtype == 'uint8':
#         test[col]=test[col].astype("int64")

In [ ]:
# test['city_code']=test['Delivery_person_ID'].str.split("RES", expand=True)[0]

In [ ]:
# test=test.drop(columns=["Delivery_person_ID" ],  axis=1)

In [ ]:
# test = test[imp_feat]

In [ ]:
# test.info()

In [ ]:
# test = test[imp_feat]

In [ ]:
# test.info()

In [ ]:
# test = encoder.transform(test)

In [ ]:
# test.head()

In [ ]:
# reg_pred_test = reg.predict(test)  # Linear Regression
# lgb_pred_test = lgb.predict(test)  # LightGBM
# cat_pred_test = cat.predict(test)  # CatBoost
# xgb_pred_test = xgb_model.predict(test)  # XGBoost

# test['time_taken'] = (
#     reg_pred_test * 0.2 +      # Trọng số của Linear Regression
#     lgb_pred_test * 0.45 +     # Trọng số của LightGBM
#     cat_pred_test * 0.35       # Trọng số của CatBoost
# )

In [ ]:
# test

In [380]:
def preprocess_data(test, encoder):
  test['Weatherconditions']=test['Weatherconditions'].str.split(" ", expand=True)[1]
  num_cols_test = [
              'Delivery_person_Age',
              'Delivery_person_Ratings',
              'Restaurant_latitude',
              'Restaurant_longitude',
              'Delivery_location_latitude',
              'Delivery_location_longitude',
              'Vehicle_condition',
              'multiple_deliveries',]
  for col in num_cols_test:
      test[col]=test[col].astype('float64')
  test['Order_Date']=pd.to_datetime(test['Order_Date'],format="%d-%m-%Y")

  test['Time_Orderd']=pd.to_timedelta(test['Time_Orderd'])
  test['Time_Order_picked']=pd.to_timedelta(test['Time_Order_picked'])

  test['Time_Order_picked_formatted']=np.where(test['Time_Order_picked'] < test['Time_Orderd'],
                                               test['Order_Date'] + pd.DateOffset(1)+test['Time_Order_picked'],
                                               test['Order_Date']+test['Time_Order_picked'])
  test['Time_Ordered_formatted'] = test['Order_Date']+ test['Time_Orderd']
  test['order_prepare_time_diff_mins']=((test['Time_Order_picked_formatted']- test['Time_Ordered_formatted']).dt.total_seconds())/60

  cols=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']
  for col in cols:
      test[col]= abs(test[col])

  test['distance_diff_KM']=np.zeros(len(test))
  restaurant_cordinates_test=test[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
  delivery_location_cordinates_test=test[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

  for i in range(len(test)):
      test['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_test[i],delivery_location_cordinates_test[i])

  test["day"] = test.Order_Date.dt.day
  test["week"] = test.Order_Date.dt.isocalendar().week
  test["month"] = test.Order_Date.dt.month
  test["quarter"] = test.Order_Date.dt.quarter
  test["year"] = test.Order_Date.dt.year
  test["hour"] = test.Time_Orderd.dt.components['hours']
  test["dayofyear"] = test.Order_Date.dt.dayofyear
  test['day_of_week'] = test.Order_Date.dt.day_of_week.astype(int)
  test["is_month_start"] = test.Order_Date.dt.is_month_start.astype(int)
  test["is_month_end"] = test.Order_Date.dt.is_month_end.astype(int)
  test["is_quarter_start"] = test.Order_Date.dt.is_quarter_start.astype(int)
  test["is_quarter_end"] = test.Order_Date.dt.is_quarter_end.astype(int)
  test["is_year_start"] = test.Order_Date.dt.is_year_start.astype(int)
  test["is_year_end"] = test.Order_Date.dt.is_year_end.astype(int)
  test["is_leap_year"] = test.Order_Date.dt.is_leap_year.astype(int)
  test["days_in_month"] = test.Order_Date.dt.days_in_month
  test['is_weekend'] = np.where(test['day_of_week'].isin([5,6]),1,0)

  test = test.drop(columns=["ID"],axis=1)

  test['distance_diff_KM']=test['distance_diff_KM'].astype("str").str.extract('(\d+)')
  test['distance_diff_KM']=test['distance_diff_KM'].astype("int64")

  test=test.drop(columns=['Order_Date','Time_Orderd','Time_Order_picked'],axis=1)
  test=test.drop(columns=['Time_Order_picked_formatted','Time_Ordered_formatted'])

  test['week']=test['week'].astype("int64")

  for col in test.columns:
    if test[col].dtype == 'uint8':
        test[col]=test[col].astype("int64")

  test['city_code']=test['Delivery_person_ID'].str.split("RES", expand=True)[0]

  test=test.drop(columns=["Delivery_person_ID" ],  axis=1)

  test = test[imp_feat]
  test = encoder.transform(test)
  return test

In [381]:
def predict(test, encoder):
    reg_pred_test = reg.predict(test)  # Linear Regression
    lgb_pred_test = lgb.predict(test)  # LightGBM
    cat_pred_test = cat.predict(test)  # CatBoost
    xgb_pred_test = xgb_model.predict(test)  # XGBoost
    test['time_taken'] = (
        reg_pred_test * 0.2 +      # Trọng số của Linear Regression
        lgb_pred_test * 0.45 +     # Trọng số của LightGBM
        cat_pred_test * 0.35       # Trọng số của CatBoost
    )
    return test


In [382]:
new_data = {
    "ID": ["0x4607"],
    "Delivery_person_ID": ["INDORES13DEL02"],
    "Delivery_person_Age": ["37"],
    "Delivery_person_Ratings": ["4.9"],
    "Restaurant_latitude": [22.745049],
    "Restaurant_longitude": [75.892471],
    "Delivery_location_latitude": [22.765049],
    "Delivery_location_longitude": [75.912471],
    "Order_Date": ["19-03-2022"],
    "Time_Orderd": ["11:30:00"],
    "Time_Order_picked": ["11:45:00"],
    "Weatherconditions": ["conditions Sunny"],
    "Road_traffic_density": ["High"],
    "Vehicle_condition": [2],
    "Type_of_order": ["Snack"],
    "Type_of_vehicle": ["motorcycle"],
    "multiple_deliveries": ["0"],
    "Festival": ["No"],
    "Delivery_person_Ratings_missing" :[0],
    "Delivery_person_Age_missing"  :[0],
    "Time_Orderd_missing"  :[0],
    "Weatherconditions_missing"  :[0],
    "City": ["Urban"],
}

# Tạo DataFrame
test = pd.DataFrame(new_data)
test = preprocess_data(test, encoder)

In [383]:
test = predict(test, encoder)

In [384]:
test.head()

,Delivery_person_Ratings,Weatherconditions,Road_traffic_density,Vehicle_condition,multiple_deliveries,Festival,City,distance_diff_KM,Delivery_person_Age_missing,Delivery_person_Ratings_missing,Time_Orderd_missing,Weatherconditions_missing,time_taken
0,4.9,21.857556,26.294607,2.0,0.0,26.294607,26.294607,3,0,0,0,0,16.102677
